In [1]:
#Load the packages
import torch
import torch.nn as nn
from lightning.pytorch import Trainer #https://lightning.ai/docs/pytorch/stable/common/trainer.html
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger
from maldi_zsl_edit.data import MALDITOFDataModule
from maldi_zsl_edit.models import ZSLClassifier

## Toy set preparation

Simulation of how the dimensions of the inputs in the ZSL work flow will change

In [2]:
#Check dimensions 
torch.manual_seed(4) #For reproducibility

#For Maldi
batch = 120
x = torch.randn(batch,6000) #(instances, features)
i = 0
x_i = x[i]

#For RNA
m = 400 #Species to be analyzed, is like a batch of seq so the 
a = torch.randint(low=0,high=5,size= (m,5,2236)) #(instances, channel, lenght), the instances are the mini batch size of sequences
j = 0
a_j = a[j]

#Species
y = torch.randint(0, m, (batch,)) #Simulate how real values will be represented

In [3]:
#Check how the embedding works
print(f"one seq\n{a}\nShape:{a.shape}\n{type(a)}") 

one seq
tensor([[[0, 3, 2,  ..., 0, 4, 0],
         [2, 3, 0,  ..., 0, 0, 3],
         [3, 3, 4,  ..., 1, 0, 1],
         [3, 3, 4,  ..., 1, 0, 3],
         [0, 1, 4,  ..., 4, 3, 2]],

        [[4, 0, 4,  ..., 0, 3, 2],
         [4, 0, 4,  ..., 0, 3, 4],
         [2, 3, 0,  ..., 4, 2, 2],
         [0, 0, 2,  ..., 0, 2, 0],
         [1, 4, 2,  ..., 3, 4, 1]],

        [[3, 3, 0,  ..., 0, 3, 0],
         [0, 2, 2,  ..., 1, 3, 3],
         [3, 2, 0,  ..., 2, 4, 4],
         [0, 1, 1,  ..., 3, 0, 4],
         [1, 4, 1,  ..., 0, 2, 0]],

        ...,

        [[0, 4, 4,  ..., 2, 2, 0],
         [1, 4, 4,  ..., 2, 4, 4],
         [2, 1, 3,  ..., 0, 3, 1],
         [0, 1, 4,  ..., 0, 2, 2],
         [4, 0, 2,  ..., 1, 1, 4]],

        [[0, 3, 2,  ..., 4, 0, 2],
         [3, 4, 3,  ..., 4, 3, 2],
         [3, 4, 4,  ..., 0, 2, 2],
         [1, 4, 0,  ..., 2, 4, 0],
         [1, 4, 2,  ..., 2, 1, 0]],

        [[3, 1, 4,  ..., 0, 3, 3],
         [2, 0, 3,  ..., 1, 3, 1],
         [3, 3, 4,  ...

In [4]:
emb = nn.Embedding(5,16) #Create it (vocab_size, hidden_sizes[0]) 
a2 = emb(a[0])#Use it
print(f"one word emb\n{a2}\nShape:{a2.shape}\n{type(a2)}")
a2.shape #The out is (channel, lenght, embedding) #The batch dissappear as the word embedding condense all that information in the embedding_dim?

one word emb
tensor([[[ 0.8584,  2.2235,  1.9713,  ...,  1.1876,  1.4561, -0.3945],
         [-0.2147, -2.1770,  0.2852,  ..., -0.0065,  0.3661, -1.4972],
         [ 0.6660,  2.1306, -0.7803,  ...,  0.6889, -0.6540, -0.0957],
         ...,
         [ 0.8584,  2.2235,  1.9713,  ...,  1.1876,  1.4561, -0.3945],
         [-1.2641, -1.0873, -0.3590,  ...,  1.9580,  2.2430, -0.3009],
         [ 0.8584,  2.2235,  1.9713,  ...,  1.1876,  1.4561, -0.3945]],

        [[ 0.6660,  2.1306, -0.7803,  ...,  0.6889, -0.6540, -0.0957],
         [-0.2147, -2.1770,  0.2852,  ..., -0.0065,  0.3661, -1.4972],
         [ 0.8584,  2.2235,  1.9713,  ...,  1.1876,  1.4561, -0.3945],
         ...,
         [ 0.8584,  2.2235,  1.9713,  ...,  1.1876,  1.4561, -0.3945],
         [ 0.8584,  2.2235,  1.9713,  ...,  1.1876,  1.4561, -0.3945],
         [-0.2147, -2.1770,  0.2852,  ..., -0.0065,  0.3661, -1.4972]],

        [[-0.2147, -2.1770,  0.2852,  ..., -0.0065,  0.3661, -1.4972],
         [-0.2147, -2.1770,  0.2

torch.Size([5, 2236, 16])

In [5]:
#MLP-CNN
e = 560 #Embedding dim
#phi = torch.randn(e,1) #This is the desired dim but is going to be out like this? maybe requiers one reshape for the shared space 
#considering that the embedding replace the feathures maybe is (1,e)
phi = torch.randn(batch,e)
pi = phi
#theta = torch.randn(e,1,m) #This is the desired dim but is going to be out like this? maybe requiers one reshape for the shared space 
#considering that the embedding replace the channels and the maxpol reduce the lenght to 1 maybe is (m,e,1)
theta = torch.randn(m,e,1)
psi = theta.reshape(-1,e)

In [6]:
score = torch.matmul(pi,psi.t())
score #use the score for the treshold, as it is not affected for the number of sequences

tensor([[ 13.4205,  -9.5598,  20.1351,  ..., -29.9532,  27.4057,  23.9957],
        [ 28.6823,  18.8329,   1.5178,  ..., -53.5264, -25.7394, -29.1681],
        [  1.3139,  31.2829, -49.6960,  ...,  18.5240, -28.2222,  12.1859],
        ...,
        [-21.9713,  10.7795,  12.7271,  ...,  33.6927,  -7.5741,   1.9863],
        [ -1.3287,  25.8044,  12.6863,  ...,  42.0087,  14.7406,  -3.5138],
        [ -0.1716, -13.7011, -48.8471,  ...,  -2.3619, -14.2327, -22.2840]])

In [7]:
#Use the softmax to get the probabilities 
softmax = nn.Softmax(dim=1) #Select the dimension that is going to be summed
prob = softmax(score) 
print(prob.shape) # (Batch of maldi,bacth of seq)
prob #The prob depends on the minibatch size, as is the number of samples.

torch.Size([120, 400])


tensor([[6.9718e-27, 7.2964e-37, 5.7471e-24,  ..., 1.4013e-45, 8.2612e-21,
         2.7296e-22],
        [3.7453e-16, 1.9768e-20, 5.9716e-28,  ..., 0.0000e+00, 8.6788e-40,
         2.8144e-41],
        [3.9107e-31, 4.0515e-18, 0.0000e+00,  ..., 1.1655e-23, 5.8855e-44,
         2.0602e-26],
        ...,
        [1.7096e-41, 2.8602e-27, 2.0056e-26,  ..., 2.5556e-17, 3.0585e-35,
         4.3407e-31],
        [1.1469e-26, 6.9709e-15, 1.4001e-20,  ..., 7.5986e-08, 1.0923e-19,
         1.2900e-27],
        [2.9661e-25, 3.9480e-31, 0.0000e+00,  ..., 3.3185e-26, 2.3203e-31,
         7.3941e-35]])

In [8]:
#Try the argmax scores, also a threshold
#prob = torch.tensor(([0.4,0.3,0.3],[0.3,0.4,0.3])) #To check how prob will work
threshold = 0.5 #note the treshold needs to be adaptative, and is more likely to be a condision, if there is
#no probability on the data set that is signicantly higher (2std) then you can not say you know the answer
best = torch.argmax(prob, dim=1)
print(best)
prob_new = torch.where(torch.any((prob > threshold), dim=1), best, torch.tensor(-1)) #If there is no value higher than threshold, then the best index is -1 (sign that there is no better answer)
print(prob_new)

tensor([ 82, 331, 211, 146,  35, 335,  52, 125, 187, 379, 337,   2,  65,  55,
        380, 394, 346, 226, 342, 100, 382, 139, 147, 104, 166, 381,  21, 103,
         64,  25, 198,  81, 116, 232, 379,  96, 208, 180, 332, 392, 141, 259,
         34, 239,  92, 374, 205, 140, 257,  42, 181, 385,  73, 268,  66, 359,
         55, 284, 373, 152, 350,   4, 146,  50,  37, 167, 253,  31,  77, 260,
        298, 301, 147, 160, 238, 166, 379,  81, 270,   1, 269,   0, 326, 278,
        271, 303, 247,  15, 205,  28, 221, 232, 384, 282, 203, 341, 103,  85,
        232, 251, 225, 193, 262, 181,  42,  11, 247, 119,  10,  73, 272, 152,
        270, 329, 375,   3, 194, 295, 116,  38])
tensor([ 82, 331, 211, 146,  35, 335,  52, 125, 187, 379, 337,   2,  -1,  55,
        380, 394, 346, 226, 342, 100, 382, 139, 147, 104, 166, 381,  21,  -1,
         64,  25, 198,  81, 116, 232, 379,  96, 208, 180, 332, 392, 141, 259,
         34, 239,  92, 374, 205, 140,  -1,  42, 181, 385,  73, 268,  66, 359,
         55, 28

## Data exploration

In [2]:
#Load the data set
dm = MALDITOFDataModule( #Personalized lightning data modules
    "Data/zsl_binned_new.h5t", #The old has problems on split
    zsl_mode = True, # False: multi-class CLF, True: ZSL
    split_index = 0, # independent train-val-test split numbered 0-9
    batch_size = 16, # important hyperparameter
    n_workers = 2, # you can leave this always if you are not CPU limited
    in_memory = True, # you can leave this always if memory is no problem
    )

dm.setup(None)


 create index train



In [4]:
#Train data exploration
batch = next(iter(dm.train_dataloader())) #Now train only has the seq of the species used in train 
print(f"Information inside the batch {list(batch)}")
print(f"The 'x' input info is in 'intensity', like {batch['intensity'][0]}, with a type {type(batch['intensity'][0])} and shape of {batch['intensity'].shape}" )
print(f"The 'a' attribute info is in 'seq', like {batch['seq'][0]}, with a type {type(batch['seq'][0])} and shape of {batch['seq'].shape}")
#Note: The info for a is dim 463, as the others are as ZSL
print(f"The 'y' real output info is in 'strain', like {batch['strain'][0]}, with a type {type(batch['strain'][0])} and shape of {batch['strain'].shape}")


 create values train

 create values train



 create values train

 create values train




/home/jorge/anaconda3/envs/Thesis/lib/python3.11/site-packages/torch/cuda/__init__.py:190: UserWarning: 
    Found GPU2 NVIDIA Tesla K40c which is of cuda capability 3.5.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability supported by this library is 3.7.
    
  warnings.warn(



 create values train

Information inside the batch ['intensity', 'mz', 'group', 'strain', 'seq', 'seq_names', 'seq_ohe']
The 'x' input info is in 'intensity', like tensor([4.1649e-05, 3.9970e-05, 1.3873e-05,  ..., 2.2899e-06, 2.0047e-07,
        1.0416e-05]), with a type <class 'torch.Tensor'> and shape of torch.Size([16, 6000])
The 'a' attribute info is in 'seq', like tensor([0, 0, 0,  ..., 0, 0, 0], dtype=torch.int32), with a type <class 'torch.Tensor'> and shape of torch.Size([463, 2236])
The 'y' real output info is in 'strain', like 204, with a type <class 'torch.Tensor'> and shape of torch.Size([16])


In [11]:
#Val data exploration
batch = next(iter(dm.val_dataloader())) #Now val has all the seq... maybe is bad as we dont have zsl on test set 
print(f"Information inside the batch {list(batch)}")
print(f"The 'x' input info is in 'intensity', like {batch['intensity'][0]}, with a type {type(batch['intensity'][0])} and shape of {batch['intensity'].shape}" )
print(f"The 'a' attribute info is in 'seq', like {batch['seq'][0]}, with a type {type(batch['seq'][0])} and shape of {batch['seq'].shape}")
print(f"The 'y' real output info is in 'strain', like {batch['strain'][0]}, with a type {type(batch['strain'][0])} and shape of {batch['strain'].shape}")

Information inside the batch ['intensity', 'mz', 'group', 'strain', 'seq', 'seq_names']
The 'x' input info is in 'intensity', like tensor([1.1096e-04, 3.1383e-04, 2.5239e-04,  ..., 2.6381e-05, 2.9734e-05,
        2.9924e-05]), with a type <class 'torch.Tensor'> and shape of torch.Size([16, 6000])
The 'a' attribute info is in 'seq', like tensor([0, 0, 0,  ..., 0, 0, 0], dtype=torch.int32), with a type <class 'torch.Tensor'> and shape of torch.Size([788, 2236])
The 'y' real output info is in 'strain', like 153, with a type <class 'torch.Tensor'> and shape of torch.Size([16])


## Training

In [3]:
#Now there should be a batch instance ["seq_ohe]", replace the batch["seq"] with it in the models file
dim_emb = 520
batch = next(iter(dm.train_dataloader()))
n_species = 788 #batch['strain'].shape[0] #Number the seq considered for the train

model = ZSLClassifier(
    mlp_kwargs = { #specify the parameters to buld the MLP ()
        'n_inputs' : 6000, #Bins of the spectra
        'emb_dim' : dim_emb, #This is the output of the branch
        'layer_dims': [512, 256],
        'layer_or_batchnorm' : "layer",
        'dropout' : 0.2,
    },
    cnn_kwargs= { #specify the parameters to buld the CNN ()
        'vocab_size' : 6, #Number of words, in this case is 5 as (A,T,C,G,-)
        'emb_dim' : dim_emb, #This is the output of the branch
        'hidden_sizes' : [64, 128], #Out chanels of the convolutions, the first work as embeding dimension
        #Note: The first hidden state is the embedding dim of the seq language processing and need to be optimized
        #Note2: The last is the embedding dim for the shared space and score function
        'blocks_per_stage' : 2, #How many residual blocks are applied before the pooling
        'kernel_size' : 7,
        'dropout' : 0.2,
    },
    n_classes = n_species,
    lr=1e-4, # important to tune
    weight_decay=0, # this you can keep constant
    lr_decay_factor=1.00, # this you can keep constant
    warmup_steps=250, # this you can keep constant
    nlp = False #Try
)


 create values train

 create values train



 create values train


 create values train



/home/jorge/anaconda3/envs/Thesis/lib/python3.11/site-packages/torch/cuda/__init__.py:190: UserWarning: 
    Found GPU2 NVIDIA Tesla K40c which is of cuda capability 3.5.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability supported by this library is 3.7.
    
  warnings.warn(



 create values train



In [4]:
#Save and monitor training with tensor board
val_ckpt = ModelCheckpoint(monitor="val_acc", mode="max")
callbacks = [val_ckpt, EarlyStopping(monitor="val_acc", patience=10, mode="max")]
logger = TensorBoardLogger("logs_folder", name="zsl_train_try") # Ctrl+Shift+P # Main folder where the training is saved and the name for the training

#Training specification
trainer = Trainer(
    max_epochs = 100, 
    accelerator='gpu', 
    strategy='auto',
    callbacks=callbacks,
    logger=logger,
    devices=[0]) #You can define epochs and training devices (look on documentation)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [5]:
#Start training
print("\n--- Start training ---\n")
trainer.fit(model, dm.train_dataloader(), dm.val_dataloader()) #Important: normally you can use only dm, but here we specify as the dim of a are different for train and val 
#Note: The model object specify what is considered an input values and what is considered an input/output value during the training on the training step method


--- Start training ---



2024-07-08 22:00:01.709501: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-07-08 22:00:02.637891: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name              | Type               | Params
---------------------------------------------------------
0 | spectrum_embedder | MLPEmbedding       | 3.6 M 
1 | seq_embedder      | CNNEmbedding       | 372 K 
2 | accuracy          | MulticlassAccuracy | 0     
3 | top5_accuracy     | MulticlassAccuracy | 0     
---------------------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
15.935    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/jorge/anaconda3/envs/Thesis/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/jorge/anaconda3/envs/Thesis/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:   0%|          | 0/1054 [00:00<?, ?it/s] 
 create values train


 create values train


 create values train


 create values train


 create values train

Epoch 0:   0%|          | 1/1054 [00:00<08:44,  2.01it/s, v_num=39]
 create values train

Epoch 0:   0%|          | 2/1054 [00:00<04:28,  3.92it/s, v_num=39]
 create values train

Epoch 0:   0%|          | 3/1054 [00:00<03:03,  5.73it/s, v_num=39]
 create values train

Epoch 0:   0%|          | 4/1054 [00:00<02:21,  7.42it/s, v_num=39]
 create values train

Epoch 0:   0%|          | 5/1054 [00:00<01:56,  9.01it/s, v_num=39]
 create values train

Epoch 0:   1%|          | 6/1054 [00:00<01:48,  9.66it/s, v_num=39]
 create values train

Epoch 0:   1%|          | 7/1054 [00:00<01:43, 10.14it/s, v_num=39]
 create values train

Epoch 0:   1%|          | 8/1054 [00:01<02:17,  7.59it/s, v_num=39]
 create values train

Epoch 0:   1%|          | 9/1054 [00:01<02:50,  6.13it/s, v_num=39]
 create values train

Epoch 0:   1%|          |

## Save the model

In [6]:
#Save the model
from datetime import datetime
timenow = datetime.now()
strtime = timenow.strftime("%Y-%m-%d_%H-%M-%S")
sure = True
if sure:
    torch.save(model, f'SavedModels\ZSLmodel{strtime}.pth')

# Test model

In [ ]:
#Load the model
sure = False
if sure:
    model = torch.load(f'SavedModels\ZSLmodel{strtime}.pth')

## Get predictions

In [17]:
#Manual calculation of the predictions
import torch.nn as nn
import torch
y_pred = torch.empty((0,788)) #the second is the number of species
y_real= []
with torch.no_grad():
    for minibatch in iter(dm.val_dataloader()):
        y_hat = model(minibatch)
        y_pred = torch.cat((y_pred,y_hat),dim=0)
        y_real+= list(minibatch['strain'])
print(y_pred.shape) #(batch size, total possible species)
y_pred

torch.Size([5941, 788])


tensor([[ -9.5106,  -9.0001,  -8.5861,  ...,  -9.8726, -18.4276, -18.4372],
        [ -2.2212,  -1.7230,  -2.1667,  ...,   7.3582,   0.9129,  -0.4655],
        [-43.8701, -41.5515, -42.6337,  ..., -18.4902, -31.7741, -33.7759],
        ...,
        [ 17.6960,  21.2303,  22.3797,  ...,   7.0832,  -9.5825,  -8.1854],
        [ 23.7911,  22.2773,  23.3971,  ...,  11.8702,  -2.3161,  -2.4400],
        [-17.0578, -16.3447, -13.2491,  ..., -21.9816, -31.0825, -33.9142]])

## Labels for Multilevel evaluation

In [20]:
pred_ind = torch.argmax(y_pred, axis=1)
real_ind = y_real
levels = ["Family", "Genus", "Species", "Strain"]
granularity_lvl = len(levels) 

In [25]:
filos = minibatch["seq_names"]

In [26]:
#Get the multilevel predictions, consider how the data is encoded (genus, species, strain)
ml_real = []
ml_pred = []
for i in range(len(y_real)):
  #for real:
  s_real = filos[real_ind[i]].split(";")
  ml_real.append(s_real)
  #for pred:
  s_pred = filos[pred_ind[i]].split(";")
  ml_pred.append(s_pred)

In [27]:
#Get them on the right format
import numpy as np
ml_real = np.array(ml_real).T
ml_pred = np.array(ml_pred).T
#List for better iteratation
ml_reals = ml_real.tolist()
ml_preds = ml_pred.tolist()

In [28]:
#Get all the possible multilevel labels
ml_level = []
for i in range(len(filos)):
  s_level = filos[i].split(";")
  ml_level.append(s_level)
ml_level = np.array(ml_level).T
ml_levels = ml_level.tolist()

In [42]:
#Total number of labels
for i in range(granularity_lvl):
    n = len(list(set(ml_levels[i])))
    print(f"For {levels[i]} there are {n} different labels")

For Family there are 77 different labels
For Genus there are 162 different labels
For Species there are 679 different labels
For Strain there are 788 different labels


## Accuracy evaluation

In [29]:
from sklearn.metrics import accuracy_score #There is also a torch version, consider it
from sklearn.preprocessing import LabelEncoder
from torchmetrics import Accuracy

#Create a accuracy evaluator
def accu_score(y_true, y_pred, level_lab):
    label_encoder = LabelEncoder()
    y_true_encoded = label_encoder.fit_transform(level_lab)
    y_true_encoded = label_encoder.transform(y_true)
    y_pred_encoded = label_encoder.transform(y_pred)

    #Using sklearn
    #accu = accuracy_score(y_true_encoded, y_pred_encoded, normalize=True) #The normalize True = number of correct predictions, False = fraction of correct predictions
    
    #Using torch
    accu = Accuracy(task="multiclass", num_classes=len(set(level_lab))) 
    accu = accu(torch.tensor(y_pred_encoded), torch.tensor(y_true_encoded))
    
    return accu

In [30]:
# run accu for each level of complexity
accu_levels = []
for level in range(granularity_lvl ):
  accu_levels.append(accu_score(ml_reals[level], ml_preds[level], ml_levels[level]))

In [32]:
# see the results
for i in range(granularity_lvl ):
  print(f"For the level {levels[i]} the accu score is: {accu_levels[i]}") 

For the level Family the accu score is: 0.6426527500152588
For the level Genus the accu score is: 0.5913146138191223
For the level Species the accu score is: 0.2295909821987152
For the level Strain the accu score is: 0.08298265933990479


## F1 evaluation

In [43]:
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from torchmetrics import F1Score

#Create an F1 evaluator
def f1_macro_score(y_true, y_pred, level_lab): #micro average is basically accuracy
    label_encoder = LabelEncoder()
    y_true_encoded = label_encoder.fit_transform(level_lab)
    y_true_encoded = label_encoder.transform(y_true)
    y_pred_encoded = label_encoder.transform(y_pred)

    #Using sklearn
    #f1_scores = f1_score(y_true_encoded, y_pred_encoded, average=None)
    #macro_f1 = sum(f1_scores) / len(f1_scores)

    #Using torch
    macro_f1 = F1Score(task="multiclass", num_classes=len(set(level_lab)), average='macro') 
    macro_f1 = macro_f1(torch.tensor(y_pred_encoded), torch.tensor(y_true_encoded))

    return macro_f1

In [44]:
# run f1_macro_score for each level of complexity
F1_levels = []
for level in range(granularity_lvl ):
  F1_levels.append(f1_macro_score(ml_reals[level], ml_preds[level], ml_levels[level]))

In [45]:
# see the results
for i in range(granularity_lvl ):
  print(f"For the level {levels[i]} the F1 score is: {F1_levels[i]}") #The predictions are no the same as the output, maybe F1 is not used there

For the level Family the F1 score is: 0.16827408969402313
For the level Genus the F1 score is: 0.13096652925014496
For the level Species the F1 score is: 0.057585593312978745
For the level Strain the F1 score is: 0.025716444477438927
